In [ ]:
!pip3 install bertopic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 47.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 21.3 MB/s eta 0:00:00
  Using cached Cython-0.29.37-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.manylinux_2_24_x86_64.whl (1.9 MB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 8.9 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (7

In [ ]:
import pandas as pd

In [ ]:
#For You dfs
con_fy = pd.read_csv("https://raw.githubusercontent.com/bensaldich/ddps_final/main/ddps/DDPS%20Final%20Project%20-%20%40thom_fy.csv")
lib_fy = pd.read_csv("https://raw.githubusercontent.com/bensaldich/ddps_final/main/ddps/DDPS%20Final%20Project%20-%20%40jsmith_fy.csv")
noone_fy = pd.read_csv("https://raw.githubusercontent.com/bensaldich/ddps_final/main/ddps/DDPS%20Final%20Project%20-%20no_one_fy.csv")
everyone_fy = pd.read_csv("https://raw.githubusercontent.com/bensaldich/ddps_final/main/ddps/DDPS%20Final%20Project%20-%20everyone_fy.csv")

# Trump OR Biden search result dfs. Data captured at the same time, in same location
lib_trump_biden = pd.read_csv("https://raw.githubusercontent.com/bensaldich/ddps_final/main/ddps/DDPS%20Final%20Project%20-%20%40JSmith_TorB.csv")
con_trump_biden = pd.read_csv("https://raw.githubusercontent.com/bensaldich/ddps_final/main/ddps/DDPS%20Final%20Project%20-%20%40Thom_TorB.csv")

#immigration OR immigrant OR migration OR migrant search results dfs. Data for lib+con captured at same
con_immi = pd.read_csv("https://raw.githubusercontent.com/bensaldich/ddps_final/main/ddps/DDPS%20Final%20Project%20-%20%40Thom_immigration.csv")
lib_immi = pd.read_csv("https://raw.githubusercontent.com/bensaldich/ddps_final/main/ddps/DDPS%20Final%20Project%20-%20%40jsmith_immigration.csv")

#immigration OR immigrant OR migration OR migrant search results dfs. Data for noone + everyone captured at same
noone_immi = pd.read_csv("https://raw.githubusercontent.com/bensaldich/ddps_final/main/ddps/DDPS%20Final%20Project%20-%20no_one_immigration.csv")
everyone_immi = pd.read_csv("https://raw.githubusercontent.com/bensaldich/ddps_final/main/ddps/DDPS%20Final%20Project%20-%20everyone_immigration.csv")

#Gaza OR Palestine OR Israel
lib_gaza = pd.read_csv("https://raw.githubusercontent.com/bensaldich/ddps_final/main/ddps/lib_gaza.csv")
con_gaza = pd.read_csv("https://raw.githubusercontent.com/bensaldich/ddps_final/main/ddps/con_gaza.csv")

In [ ]:
df = lib_immi

In [ ]:
df['body'][15]

'Welcome to the Immigrant Dad Talk Show https://t.co/4vb11nwTX5'

In [ ]:
df['body'].fillna("",inplace=True)

In [ ]:
# Removing URLs from the body of tweets
df['body2'] = df['body'].replace(r'http\S+', '', regex=True).replace(r'www\S+', '', regex=True)

In [ ]:
# Removing mentions from the body of tweets
df['body2'] = df['body2'].str.replace('@[A-Za-z0-9]+\s?', '', regex=True)

In [ ]:
df['body2'][15]

'Welcome to the Immigrant Dad Talk Show '

In [ ]:
documents = list(df['body2'])[:2000]

In [ ]:
documents[15]

'Welcome to the Immigrant Dad Talk Show '

In [ ]:
from bertopic import BERTopic
from sentence_transformers import SentenceTransformer
from umap import UMAP
from hdbscan import HDBSCAN
from sklearn.feature_extraction.text import CountVectorizer
#from sklearn.decomposition import LatentDirichletAllocation

In [ ]:
## first, embed the documents
model = SentenceTransformer("all-MiniLM-L6-v2")
document_vectors = model.encode(documents,show_progress_bar=True)

Batches:   0%|          | 0/48 [00:00<?, ?it/s]

In [ ]:
umap_model = UMAP(n_neighbors=20, n_components=5, min_dist=0.01, metric='cosine', random_state=42)
hdbscan_model = HDBSCAN(min_cluster_size=25, min_samples=4, metric='euclidean', cluster_selection_method='eom')

In [ ]:
vectorizer = CountVectorizer(stop_words='english')

In [ ]:
# Training our topic model with BERTopic
topic_model = BERTopic(
    embedding_model=model,
    umap_model=umap_model,
    hdbscan_model=hdbscan_model,
    vectorizer_model=vectorizer,
    verbose=True
).fit(documents, document_vectors)

2024-05-13 20:07:00,646 - BERTopic - Dimensionality - Fitting the dimensionality reduction algorithm
2024-05-13 20:07:08,794 - BERTopic - Dimensionality - Completed ✓
2024-05-13 20:07:08,796 - BERTopic - Cluster - Start clustering the reduced embeddings
2024-05-13 20:07:08,849 - BERTopic - Cluster - Completed ✓
2024-05-13 20:07:08,856 - BERTopic - Representation - Extracting topics from clusters using representation models.
2024-05-13 20:07:08,941 - BERTopic - Representation - Completed ✓


In [ ]:
labels = list(df['body2'])

In [ ]:
labels[1]


'How to Fill In Wrinkles Using This Everyday Household Item, This at home remedy for aging wrinkles doesn’t require cream. Watch Now⬇️ '

In [ ]:
topic_model.get_representative_docs(5)

['I realised how racist much of America is when an immigrant of questionable status and a nude modelling past was adored by many because she was white. But an intelligent, qualified all American women was abused for years and compared to an ape and a man because she was black…',
 'Most of us grew up around other Black-Americans, not black immigrant children. I believe a lot them are lying about being "bullied" by us. A lot us don\'t come into contact with black immigrants unless we go to places where they have a big immigrant population such as NYC.',
 'Not an immigrant.']

In [ ]:
topic_model.get_topic_info()

,Topic,Count,Name,Representation,Representative_Docs
0,-1,478,-1_immigration_migrant_immigrant_illegal,"[immigration, migrant, immigrant, illegal, mig...",[Ex-immigration minister Robert Jenrick propos...
1,0,194,0_illegal_immigrant_migrant_arrested,"[illegal, immigrant, migrant, arrested, tiktok...",[Follow up\nAn illegal immigrant TikTok influe...
2,1,150,1_migration_pi_amp_customer,"[migration, pi, amp, customer, choice, game, g...",[We're thrilled to announce that HCLTech has b...
3,2,132,2_immigration_uk_labour_migration,"[immigration, uk, labour, migration, britain, ...","[Reform UK want 'Net Zero' immigration, which ..."
4,3,106,3_ireland_eu_irish_pact,"[ireland, eu, irish, pact, asylum, migration, ...",[🔥 BREAKING: Irish teenagers are protesting ri...
5,4,82,4_migrant_nyc_crisis_chicago,"[migrant, nyc, crisis, chicago, homeless, city...",[“This country has now collected all the garba...
6,5,71,5_immigrant_black_white_brown,"[immigrant, black, white, brown, immigration, ...",[I realised how racist much of America is when...
7,6,70,6_border_immigration_senate_texas,"[border, immigration, senate, texas, voted, ma...","[ABC: ""Did you watch that border video down in..."
8,7,52,7_immigration_mass_need_people,"[immigration, mass, need, people, housing, bui...",[Derp... migration literally does all those th...
9,8,37,8_biden_joe_president_illegal,"[biden, joe, president, illegal, trump, immigr...",[President Donald J. Trump\nTRUTH SOCIAL POST\...


In [ ]:
topic_model.visualize_barchart(n_words=15,top_n_topics=25)

In [ ]:
topic_model.visualize_documents(labels, embeddings=document_vectors)